In [1]:
import pdfplumber
import os

In [23]:
input_dir = "../output/pdf_download_list/pdfs/"
files = os.listdir(input_dir)
court_summaries_commonpleas = [f for f in files if 'CS_CP' in f]
court_summaries_magisterial = [f for f in files if 'CS_MJ' in f]

In [ ]:
pdf_path = "../output/pdf_download_list/pdfs/CS_CP-51-CR-0013145-2010.pdf"
pdf = pdfplumber.open(pdf_path)
pages = pdf.pages
for page in pages:
    text = page.extract_text().split("\n")
    
    # Beginning part of every court summary in court of common pleas has a block of text with person information.
    poi_start_index = [i for i, x in enumerate(text) if "DOB:" in x][0]
    poi_end_index = [i for i,x in enumerate(text) if "closed" in x.lower() or "inactive" in x.lower() or "active" in x.lower()][0]
    poi = text[poi_start_index:poi_end_index]

    # Name, DOB, and Sex appear on the first line.
    name = poi[0].split("DOB:")[0].strip()
    dob = poi[0].split("DOB:")[1].split("Sex:")[0].strip()
    sex = poi[0].split("DOB:")[1].split("Sex:")[1].strip()

    # Location and Eye Color appear on the second line.
    location = poi[1].split("Eyes:")[0].strip()
    eyes = poi[1].split("Eyes:")[1].strip()

    # Alias and hair color are on the third line, but alias is blank on this line.
    hair = poi[2].split("Hair:")[1].strip()

    # The first alias and race are on the fourth line.
    alias = poi[3].split("Race:")[0].strip()
    race = poi[3].split("Race:")[1].strip()                          

    # The rest of the aliases are on subequent lines.
    remainder_alias = poi[4:len(poi)]
    remainder_alias.append(alias)

    print("Name: " + name)
    print("DOB: " + dob)
    print("Sex: " + sex)
    print("Location: " + location)
    print("Eyes: " + eyes)
    print("Hair: " + hair)
    print("Race: " + race)
    print(remainder_alias)

Name: Johnson, Edward
DOB: 09/28/1988
Sex: Male
Location: Philadelphia, PA 19141
Eyes: Brown
Hair: Black
Race: Black
None


IndexError: list index out of range